In [1]:
from selenium import webdriver
import chromedriver_autoinstaller
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re


In [12]:
def amazon_crawling(soup, m):
    
    tmp_lst = []
    
    for x in range(m):
        try:
            prd_name = soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small > div.a-section.a-spacing-none.a-spacing-top-small.s-title-instructions-style > h2')[x].text
        except IndexError:
            prd_name = None
            
        try:
            tmp = soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small > div:nth-child(2) > div > span:nth-child(1) > span.a-size-base')[x].text
            tmps = re.sub('[()]','', tmp)
            prd_rating= float(tmps)
        except IndexError:
            prd_rating = None
        
        try:
            tmp = soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small > div.a-section.a-spacing-none.a-spacing-top-small.s-price-instructions-style > div > a > span.a-price > span.a-offscreen')[x].text
            tmps = re.sub('[$]','',tmp)
            prd_price = float(tmps)
        except IndexError:
            prd_price = None
        
        try:        
            tmp = soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small > div.a-section.a-spacing-none.a-spacing-top-small.s-title-instructions-style > h2 > a')[x].get('href')
            prd_link = f'https://www.amazon.com{tmp}'
        except IndexError:
            prd_link = None
        
        try:
            tmp = soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small > div:nth-child(2) > div > span:nth-child(2) > a > span')[x].text.replace(',', '')
            tmps = re.sub('[()]','',tmp)
            prd_reviews = int(tmps)
        except IndexError:
            prd_reviews = None
            
        tmp_lst.append([prd_name, prd_rating, prd_price, prd_link, prd_reviews])

    return tmp_lst

In [13]:
base_url = 'https://www.amazon.com/s?i=hpc-intl-ship&bbn=16225010011&rh=n%3A16225010011%2Cn%3A3764441&page={}&qid=1670091061&ref=sr_pg_3'
driver = webdriver.Chrome('/Users/jeongeuiju/Amazon_Web_Crowling/chromedriver')

total_lst = []

for n in range(400) :
    url = base_url.format(n+1)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    tmp_lst2 = amazon_crawling(soup, 12)
    total_lst.extend(tmp_lst2)

/var/folders/ss/fbwhhkzj2_99l0jshybs2tm80000gn/T/ipykernel_4545/3567269092.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/jeongeuiju/Amazon_Web_Crowling/chromedriver')


In [14]:
df = pd.DataFrame(total_lst, columns = ['Name', 'Ratings', 'Price', 'Link', 'Reviews'])

df.to_csv('Crawling_Amazon.csv', index_label = False)